In [ ]:
!pip install kaggle

In [ ]:
import os
import json
from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding , LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
with ZipFile("IMDB Dataset.csv.zip","r") as zip_ref:
  zip_ref.extractall()

In [ ]:
!ls


'IMDB Dataset.csv'  'IMDB Dataset.csv.zip'   sample_data


In [ ]:
 data = pd.read_csv("/content/IMDB Dataset.csv")

In [ ]:
data.shape

(50000, 2)

In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [ ]:
data["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [ ]:
data.replace({"sentiment":{"positive":1,"negative":0}},inplace=True)

/tmp/ipython-input-3877186056.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment":{"positive":1,"negative":0}},inplace=True)


In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
data["sentiment"].value_counts()

,count
sentiment,
1,25000
0,25000


In [ ]:
train_data,test_data = train_test_split(data,test_size = 0.2, random_state = 42)

In [ ]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


In [ ]:
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen= 200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen= 200)

In [ ]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [ ]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [ ]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [ ]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


In [ ]:
print(Y_test)

33553    1
9427     1
199      0
12447    1
39489    0
        ..
28567    0
25079    1
18707    1
15200    0
5857     1
Name: sentiment, Length: 10000, dtype: int64


LSTM - Long Short Term Memory

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=5000,output_dim=128,input_length=200))
model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation="sigmoid"))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.build(input_shape=(None, 200))
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ (None, 200, 128)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [ ]:
model.fit(X_train,Y_train,epochs=5,batch_size=64,validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 206s 406ms/step - accuracy: 0.7278 - loss: 0.5320 - val_accuracy: 0.8449 - val_loss: 0.3835
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 203s 407ms/step - accuracy: 0.8562 - loss: 0.3462 - val_accuracy: 0.8570 - val_loss: 0.3432
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 204s 408ms/step - accuracy: 0.8667 - loss: 0.3262 - val_accuracy: 0.8595 - val_loss: 0.3307
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 202s 405ms/step - accuracy: 0.8931 - loss: 0.2738 - val_accuracy: 0.8739 - val_loss: 0.3209
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 202s 404ms/step - accuracy: 0.9086 - loss: 0.2388 - val_accuracy: 0.8754 - val_loss: 0.3172


Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 203s 405ms/step - accuracy: 0.9159 - loss: 0.2120 - val_accuracy: 0.8774 - val_loss: 0.3195
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 202s 405ms/step - accuracy: 0.9280 - loss: 0.1898 - val_accuracy: 0.8781 - val_loss: 0.3406
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 201s 403ms/step - accuracy: 0.9354 - loss: 0.1673 - val_accuracy: 0.8745 - val_loss: 0.3426
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 202s 404ms/step - accuracy: 0.9475 - loss: 0.1421 - val_accuracy: 0.8681 - val_loss: 0.3599
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 202s 404ms/step - accuracy: 0.9510 - loss: 0.1326 - val_accuracy: 0.8745 - val_loss: 0.3765


In [ ]:
loss , accuracy = model.evaluate(X_test,Y_test)
print("Test Loss:{loss}")
print("Test Accuracy:{accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 39s 123ms/step - accuracy: 0.8738 - loss: 0.3593
Test Loss:{loss}
Test Accuracy:{accuracy}


In [ ]:
def predict_sentiment(review):
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence,maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else " negative"
  return sentiment

In [ ]:
new_review = "This movie was fantastic. I loved it."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is : {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 419ms/step
The sentiment of the review is : positive


In [ ]:
new_review = "I am not going to suggest you to watch that movie "
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is : {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
The sentiment of the review is : positive
